In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData

from data.node_datamodule import NodeDataModule
from models.graph_embedding import GraphEmbedding

In [ ]:
def adata2data(adata:AnnData)-> Data:
    sq.gr.spatial_neighbors(adata, coord_type="generic")
    r, c = adata.obsp['spatial_connectivities'].nonzero()
    edge_index = torch.vstack([torch.from_numpy(r).to(
        torch.long), torch.from_numpy(c).to(torch.long)])
    cell_type = torch.from_numpy(pd.get_dummies(adata.obs).to_numpy())
    gene_expression = torch.from_numpy(adata.X)
    return Data(
        edge_index=edge_index,
        x=gene_expression,
        y=cell_type
    )

In [ ]:
adata = sq.datasets.imc()

In [ ]:
dm = NodeDataModule(adata=adata,adata2data_fn=adata2data, num_workers = 12, batch_size=100)

In [ ]:
model = GraphEmbedding(num_features=34,latent_dim=30,lr=0.0001,weight_decay=0.000001)

In [ ]:
trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=12,log_every_n_steps=1)

In [ ]:
trainer.fit(model,datamodule=dm)